# Задание 1.2 - Линейный классификатор (Linear classifier)

В этом задании мы реализуем другую модель машинного обучения - линейный классификатор. Линейный классификатор подбирает для каждого класса веса, на которые нужно умножить значение каждого признака и потом сложить вместе.
Тот класс, у которого эта сумма больше, и является предсказанием модели.

В этом задании вы:
- потренируетесь считать градиенты различных многомерных функций
- реализуете подсчет градиентов через линейную модель и функцию потерь softmax
- реализуете процесс тренировки линейного классификатора
- подберете параметры тренировки на практике

На всякий случай, еще раз ссылка на туториал по numpy:  
http://cs231n.github.io/python-numpy-tutorial/

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
from dataset import load_svhn, random_split_train_val
from gradient_check import check_gradient
from metrics import multiclass_accuracy 
import linear_classifer

# Как всегда, первым делом загружаем данные

Мы будем использовать все тот же SVHN.

In [3]:
def prepare_for_linear_classifier(train_X, test_X):
    train_flat = train_X.reshape(train_X.shape[0], -1).astype(np.float) / 255.0
    test_flat = test_X.reshape(test_X.shape[0], -1).astype(np.float) / 255.0
    
    # Subtract mean
    mean_image = np.mean(train_flat, axis = 0)
    train_flat -= mean_image
    test_flat -= mean_image
    
    # Add another channel with ones as a bias term
    train_flat_with_ones = np.hstack([train_flat, np.ones((train_X.shape[0], 1))])
    test_flat_with_ones = np.hstack([test_flat, np.ones((test_X.shape[0], 1))])    
    return train_flat_with_ones, test_flat_with_ones
    
train_X, train_y, test_X, test_y = load_svhn("data", max_train=10000, max_test=1000)    
train_X, test_X = prepare_for_linear_classifier(train_X, test_X)
# Split train into train and val
train_X, train_y, val_X, val_y = random_split_train_val(train_X, train_y, num_val = 1000)

# Играемся с градиентами!

В этом курсе мы будем писать много функций, которые вычисляют градиенты аналитическим методом.

Все функции, в которых мы будем вычислять градиенты, будут написаны по одной и той же схеме.  
Они будут получать на вход точку, где нужно вычислить значение и градиент функции, а на выходе будут выдавать кортеж (tuple) из двух значений - собственно значения функции в этой точке (всегда одно число) и аналитического значения градиента в той же точке (той же размерности, что и вход).
```
def f(x):
    """
    Computes function and analytic gradient at x
    
    x: np array of float, input to the function
    
    Returns:
    value: float, value of the function 
    grad: np array of float, same shape as x
    """
    ...
    
    return value, grad
```

Необходимым инструментом во время реализации кода, вычисляющего градиенты, является функция его проверки. Эта функция вычисляет градиент численным методом и сверяет результат с градиентом, вычисленным аналитическим методом.

Мы начнем с того, чтобы реализовать вычисление численного градиента (numeric gradient) в функции `check_gradient` в `gradient_check.py`. Эта функция будет принимать на вход функции формата, заданного выше, использовать значение `value` для вычисления численного градиента и сравнит его с аналитическим - они должны сходиться.

Напишите часть функции, которая вычисляет градиент с помощью численной производной для каждой координаты. Для вычисления производной используйте так называемую two-point formula (https://en.wikipedia.org/wiki/Numerical_differentiation):

![image](https://wikimedia.org/api/rest_v1/media/math/render/svg/22fc2c0a66c63560a349604f8b6b39221566236d)

Все функции приведенные в следующей клетке должны проходить gradient check.

In [4]:
# TODO: Implement check_gradient function in gradient_check.py
# All the functions below should pass the gradient check

def square(x):
    return float(x*x), 2*x

check_gradient(square, np.array([3.0]))

def array_sum(x):
    assert x.shape == (2,), x.shape
    return np.sum(x), np.ones_like(x)

check_gradient(array_sum, np.array([3.0, 2.0]))

def array_2d_sum(x):
    assert x.shape == (2,2)
    return np.sum(x), np.ones_like(x)

check_gradient(array_2d_sum, np.array([[3.0, 2.0], [1.0, 0.0]]))

Gradient check passed!
Gradient check passed!
Gradient check passed!


True

## Начинаем писать свои функции, считающие аналитический градиент

Теперь реализуем функцию softmax, которая получает на вход оценки для каждого класса и преобразует их в вероятности от 0 до 1:
![image](https://wikimedia.org/api/rest_v1/media/math/render/svg/e348290cf48ddbb6e9a6ef4e39363568b67c09d3)

**Важно:** Практический аспект вычисления этой функции заключается в том, что в ней учавствует вычисление экспоненты от потенциально очень больших чисел - это может привести к очень большим значениям в числителе и знаменателе за пределами диапазона float.

К счастью, у этой проблемы есть простое решение -- перед вычислением softmax вычесть из всех оценок максимальное значение среди всех оценок:
```
predictions -= np.max(predictions)
```
(подробнее здесь - http://cs231n.github.io/linear-classify/#softmax, секция `Practical issues: Numeric stability`)

In [5]:
# TODO Implement softmax and cross-entropy for single sample
probs = linear_classifer.softmax(np.array([-10, 0, 10]))

# Make sure it works for big numbers too!
probs = linear_classifer.softmax(np.array([1000, 0, 0]))
assert np.isclose(probs[0], 1.0)

Кроме этого, мы реализуем cross-entropy loss, которую мы будем использовать как функцию ошибки (error function).
В общем виде cross-entropy определена следующим образом:
![image](https://wikimedia.org/api/rest_v1/media/math/render/svg/0cb6da032ab424eefdca0884cd4113fe578f4293)

где x - все классы, p(x) - истинная вероятность принадлежности сэмпла классу x, а q(x) - вероятность принадлежности классу x, предсказанная моделью.  
В нашем случае сэмпл принадлежит только одному классу, индекс которого передается функции. Для него p(x) равна 1, а для остальных классов - 0. 

Это позволяет реализовать функцию проще!

In [6]:
probs = linear_classifer.softmax(np.array([-5, 0, 5]))
linear_classifer.cross_entropy_loss(probs, 1)

5.006760443547122

После того как мы реализовали сами функции, мы можем реализовать градиент.

Оказывается, что вычисление градиента становится гораздо проще, если объединить эти функции в одну, которая сначала вычисляет вероятности через softmax, а потом использует их для вычисления функции ошибки через cross-entropy loss.

Эта функция `softmax_with_cross_entropy` будет возвращает и значение ошибки, и градиент по входным параметрам. Мы проверим корректность реализации с помощью `check_gradient`.

In [7]:
# TODO Implement combined function or softmax and cross entropy and produces gradient
loss, grad = linear_classifer.softmax_with_cross_entropy(np.array([1, 0, 0]), 1)
check_gradient(lambda x: linear_classifer.softmax_with_cross_entropy(x, 1), np.array([1, 0, 0], np.float))

Gradient check passed!


True

В качестве метода тренировки мы будем использовать стохастический градиентный спуск (stochastic gradient descent или SGD), который работает с батчами сэмплов. 

Поэтому все наши фукнции будут получать не один пример, а батч, то есть входом будет не вектор из `num_classes` оценок, а матрица размерности `batch_size, num_classes`. Индекс примера в батче всегда будет первым измерением.

Следующий шаг - переписать наши функции так, чтобы они поддерживали батчи.

Финальное значение функции ошибки должно остаться числом, и оно равно среднему значению ошибки среди всех примеров в батче.

In [8]:
# TODO Extend combined function so it can receive a 2d array with batch of samples
np.random.seed(42)
# Test batch_size = 1
num_classes = 4
batch_size = 1
predictions = np.random.randint(-1, 3, size=(batch_size, num_classes)).astype(np.float)
target_index = np.random.randint(0, num_classes, size=(batch_size, 1)).astype(np.int)
check_gradient(lambda x: linear_classifer.softmax_with_cross_entropy(x, target_index), predictions)

# Test batch_size = 3
num_classes = 4
batch_size = 3
predictions = np.random.randint(-1, 3, size=(batch_size, num_classes)).astype(np.float)
target_index = np.random.randint(0, num_classes, size=(batch_size, 1)).astype(np.int)
check_gradient(lambda x: linear_classifer.softmax_with_cross_entropy(x, target_index), predictions)

# Make sure maximum subtraction for numberic stability is done separately for every sample in the batch
probs = linear_classifer.softmax(np.array([[20, 0, 0], [1000, 0, 0]]))
assert np.all(np.isclose(probs[:, 0], 1.0))

Gradient check passed!
Gradient check passed!


### Наконец, реализуем сам линейный классификатор!

softmax и cross-entropy получают на вход оценки, которые выдает линейный классификатор.

Он делает это очень просто: для каждого класса есть набор весов, на которые надо умножить пиксели картинки и сложить. Получившееся число и является оценкой класса, идущей на вход softmax.

Таким образом, линейный классификатор можно представить как умножение вектора с пикселями на матрицу W размера `num_features, num_classes`. Такой подход легко расширяется на случай батча векторов с пикселями X размера `batch_size, num_features`:

`predictions = X * W`, где `*` - матричное умножение.

Реализуйте функцию подсчета линейного классификатора и градиентов по весам `linear_softmax` в файле `linear_classifer.py`

In [9]:
# TODO Implement linear_softmax function that uses softmax with cross-entropy for linear classifier
batch_size = 2
num_classes = 2
num_features = 3
np.random.seed(42)
W = np.random.randint(-1, 3, size=(num_features, num_classes)).astype(np.float)
X = np.random.randint(-1, 3, size=(batch_size, num_features)).astype(np.float)
target_index = np.ones(batch_size, dtype=np.int)

loss, dW = linear_classifer.linear_softmax(X, W, target_index)
check_gradient(lambda w: linear_classifer.linear_softmax(X, w, target_index), W)

Gradient check passed!


True

### И теперь регуляризация

Мы будем использовать L2 regularization для весов как часть общей функции ошибки.

Напомним, L2 regularization определяется как

l2_reg_loss = regularization_strength * sum<sub>ij</sub> W[i, j]<sup>2</sup>

Реализуйте функцию для его вычисления и вычисления соотвествующих градиентов.

In [10]:
# TODO Implement l2_regularization function that implements loss for L2 regularization
linear_classifer.l2_regularization(W, 0.01)
check_gradient(lambda w: linear_classifer.l2_regularization(w, 0.01), W)

Gradient check passed!


True

### Тренировка!

Градиенты в порядке, реализуем процесс тренировки!

In [11]:
# TODO: Implement LinearSoftmaxClassifier.fit function
classifier = linear_classifer.LinearSoftmaxClassifier()
loss_history = classifier.fit(train_X, train_y, epochs=10, learning_rate=1e-3, batch_size=300, reg=1e1)

Epoch 0, loss: 2.397363
Epoch 1, loss: 2.330354
Epoch 2, loss: 2.311002
Epoch 3, loss: 2.303897
Epoch 4, loss: 2.303257
Epoch 5, loss: 2.302898
Epoch 6, loss: 2.302564
Epoch 7, loss: 2.301815
Epoch 8, loss: 2.301252
Epoch 9, loss: 2.301256


In [12]:
# Let's check how it performs on validation set
pred = classifier.predict(val_X)
accuracy = multiclass_accuracy(pred, val_y)
print("Accuracy: ", accuracy)

# Now, let's train more and see if it performs better
classifier.fit(train_X, train_y, epochs=100, learning_rate=1e-3, batch_size=300, reg=1e1)
pred = classifier.predict(val_X)
accuracy = multiclass_accuracy(pred, val_y)
print("Accuracy after training for 100 epochs: ", accuracy)

Accuracy:  0.127
Epoch 0, loss: 2.301800
Epoch 1, loss: 2.302558
Epoch 2, loss: 2.302208
Epoch 3, loss: 2.303309
Epoch 4, loss: 2.302126
Epoch 5, loss: 2.301524
Epoch 6, loss: 2.301150
Epoch 7, loss: 2.301359
Epoch 8, loss: 2.302305
Epoch 9, loss: 2.302205
Epoch 10, loss: 2.301595
Epoch 11, loss: 2.302039
Epoch 12, loss: 2.301247
Epoch 13, loss: 2.301836
Epoch 14, loss: 2.302396
Epoch 15, loss: 2.302299
Epoch 16, loss: 2.301161
Epoch 17, loss: 2.302058
Epoch 18, loss: 2.302180
Epoch 19, loss: 2.302339
Epoch 20, loss: 2.302694
Epoch 21, loss: 2.301872
Epoch 22, loss: 2.302261
Epoch 23, loss: 2.302209
Epoch 24, loss: 2.302215
Epoch 25, loss: 2.301591
Epoch 26, loss: 2.301535
Epoch 27, loss: 2.302557
Epoch 28, loss: 2.301689
Epoch 29, loss: 2.301815
Epoch 30, loss: 2.301483
Epoch 31, loss: 2.301553
Epoch 32, loss: 2.301640
Epoch 33, loss: 2.301991
Epoch 34, loss: 2.302104
Epoch 35, loss: 2.301741
Epoch 36, loss: 2.302652
Epoch 37, loss: 2.303017
Epoch 38, loss: 2.301521
Epoch 39, loss: 2.

# Как и раньше, используем кросс-валидацию для подбора гиперпараметтов.

В этот раз, чтобы тренировка занимала разумное время, мы будем использовать только одно разделение на тренировочные (training) и проверочные (validation) данные.

Теперь нам нужно подобрать не один, а два гиперпараметра! Не ограничивайте себя изначальными значениями в коде.  
Добейтесь точности более чем **20%** на проверочных данных (validation data).

In [13]:
num_epochs = 200
batch_size = 300

learning_rates = [1e-3, 1e-4, 1e-5, 1e-6]
reg_strengths = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7]

best_classifier = None
best_val_accuracy = 0

# TODO use validation set to find the best hyperparameters
# hint: for best results, you might need to try more values for learning rate and regularization strength 
# than provided initially

for lr in learning_rates:
    for rs in reg_strengths:
        classifier = linear_classifer.LinearSoftmaxClassifier()
        loss_history = classifier.fit(train_X, train_y, epochs=num_epochs,
                                      learning_rate=lr, batch_size=batch_size, reg=rs)
        pred = classifier.predict(val_X)
        accuracy = multiclass_accuracy(pred, val_y)
        if accuracy > best_val_accuracy:
            best_classifier = classifier
            best_val_accuracy = accuracy
            best_lr, best_rs = lr, rs

Epoch 0, loss: 2.301707
Epoch 1, loss: 2.300949
Epoch 2, loss: 2.302234
Epoch 3, loss: 2.300502
Epoch 4, loss: 2.297199
Epoch 5, loss: 2.297611
Epoch 6, loss: 2.293521
Epoch 7, loss: 2.297427
Epoch 8, loss: 2.291339
Epoch 9, loss: 2.289791
Epoch 10, loss: 2.290227
Epoch 11, loss: 2.290169
Epoch 12, loss: 2.291655
Epoch 13, loss: 2.294394
Epoch 14, loss: 2.290892
Epoch 15, loss: 2.282293
Epoch 16, loss: 2.289088
Epoch 17, loss: 2.281336
Epoch 18, loss: 2.286637
Epoch 19, loss: 2.283993
Epoch 20, loss: 2.280706
Epoch 21, loss: 2.281599
Epoch 22, loss: 2.283540
Epoch 23, loss: 2.284469
Epoch 24, loss: 2.285648
Epoch 25, loss: 2.280411
Epoch 26, loss: 2.276692
Epoch 27, loss: 2.281028
Epoch 28, loss: 2.280838
Epoch 29, loss: 2.287082
Epoch 30, loss: 2.277126
Epoch 31, loss: 2.278020
Epoch 32, loss: 2.272120
Epoch 33, loss: 2.273949
Epoch 34, loss: 2.271239
Epoch 35, loss: 2.276765
Epoch 36, loss: 2.267495
Epoch 37, loss: 2.281558
Epoch 38, loss: 2.274868
Epoch 39, loss: 2.271177
Epoch 40, 

Epoch 125, loss: 2.239884
Epoch 126, loss: 2.207968
Epoch 127, loss: 2.206040
Epoch 128, loss: 2.220685
Epoch 129, loss: 2.232900
Epoch 130, loss: 2.230158
Epoch 131, loss: 2.246146
Epoch 132, loss: 2.232686
Epoch 133, loss: 2.212668
Epoch 134, loss: 2.214643
Epoch 135, loss: 2.220176
Epoch 136, loss: 2.214314
Epoch 137, loss: 2.231625
Epoch 138, loss: 2.215159
Epoch 139, loss: 2.227753
Epoch 140, loss: 2.223434
Epoch 141, loss: 2.215310
Epoch 142, loss: 2.230409
Epoch 143, loss: 2.228479
Epoch 144, loss: 2.200857
Epoch 145, loss: 2.224060
Epoch 146, loss: 2.206285
Epoch 147, loss: 2.230699
Epoch 148, loss: 2.207567
Epoch 149, loss: 2.225945
Epoch 150, loss: 2.204691
Epoch 151, loss: 2.231509
Epoch 152, loss: 2.214328
Epoch 153, loss: 2.214273
Epoch 154, loss: 2.227335
Epoch 155, loss: 2.196090
Epoch 156, loss: 2.213384
Epoch 157, loss: 2.213286
Epoch 158, loss: 2.226382
Epoch 159, loss: 2.201351
Epoch 160, loss: 2.228146
Epoch 161, loss: 2.211618
Epoch 162, loss: 2.228881
Epoch 163, l

Epoch 47, loss: 2.265265
Epoch 48, loss: 2.261390
Epoch 49, loss: 2.263553
Epoch 50, loss: 2.261855
Epoch 51, loss: 2.258133
Epoch 52, loss: 2.268729
Epoch 53, loss: 2.276194
Epoch 54, loss: 2.262881
Epoch 55, loss: 2.262800
Epoch 56, loss: 2.265209
Epoch 57, loss: 2.250925
Epoch 58, loss: 2.249120
Epoch 59, loss: 2.266703
Epoch 60, loss: 2.257007
Epoch 61, loss: 2.257398
Epoch 62, loss: 2.261112
Epoch 63, loss: 2.255927
Epoch 64, loss: 2.260787
Epoch 65, loss: 2.257807
Epoch 66, loss: 2.253572
Epoch 67, loss: 2.242457
Epoch 68, loss: 2.269272
Epoch 69, loss: 2.245399
Epoch 70, loss: 2.248048
Epoch 71, loss: 2.240187
Epoch 72, loss: 2.255451
Epoch 73, loss: 2.244413
Epoch 74, loss: 2.248828
Epoch 75, loss: 2.262491
Epoch 76, loss: 2.242827
Epoch 77, loss: 2.257438
Epoch 78, loss: 2.251271
Epoch 79, loss: 2.242803
Epoch 80, loss: 2.249192
Epoch 81, loss: 2.242130
Epoch 82, loss: 2.244215
Epoch 83, loss: 2.253258
Epoch 84, loss: 2.244790
Epoch 85, loss: 2.234758
Epoch 86, loss: 2.244780


Epoch 169, loss: 2.211769
Epoch 170, loss: 2.201998
Epoch 171, loss: 2.186109
Epoch 172, loss: 2.224254
Epoch 173, loss: 2.168218
Epoch 174, loss: 2.216534
Epoch 175, loss: 2.217960
Epoch 176, loss: 2.219892
Epoch 177, loss: 2.183889
Epoch 178, loss: 2.215506
Epoch 179, loss: 2.185744
Epoch 180, loss: 2.219981
Epoch 181, loss: 2.210040
Epoch 182, loss: 2.207816
Epoch 183, loss: 2.194906
Epoch 184, loss: 2.203751
Epoch 185, loss: 2.210483
Epoch 186, loss: 2.196954
Epoch 187, loss: 2.207656
Epoch 188, loss: 2.221451
Epoch 189, loss: 2.193928
Epoch 190, loss: 2.194184
Epoch 191, loss: 2.201380
Epoch 192, loss: 2.191522
Epoch 193, loss: 2.198223
Epoch 194, loss: 2.188763
Epoch 195, loss: 2.225821
Epoch 196, loss: 2.243986
Epoch 197, loss: 2.214685
Epoch 198, loss: 2.202836
Epoch 199, loss: 2.223986
Epoch 0, loss: 2.302119
Epoch 1, loss: 2.301594
Epoch 2, loss: 2.299195
Epoch 3, loss: 2.299670
Epoch 4, loss: 2.299073
Epoch 5, loss: 2.296687
Epoch 6, loss: 2.297722
Epoch 7, loss: 2.294331
Ep

Epoch 94, loss: 2.296561
Epoch 95, loss: 2.293002
Epoch 96, loss: 2.296112
Epoch 97, loss: 2.296055
Epoch 98, loss: 2.292832
Epoch 99, loss: 2.294376
Epoch 100, loss: 2.292266
Epoch 101, loss: 2.293966
Epoch 102, loss: 2.292155
Epoch 103, loss: 2.294329
Epoch 104, loss: 2.293372
Epoch 105, loss: 2.294910
Epoch 106, loss: 2.292567
Epoch 107, loss: 2.293849
Epoch 108, loss: 2.294986
Epoch 109, loss: 2.297052
Epoch 110, loss: 2.289857
Epoch 111, loss: 2.290115
Epoch 112, loss: 2.292772
Epoch 113, loss: 2.293676
Epoch 114, loss: 2.289991
Epoch 115, loss: 2.293186
Epoch 116, loss: 2.294235
Epoch 117, loss: 2.289904
Epoch 118, loss: 2.290693
Epoch 119, loss: 2.296022
Epoch 120, loss: 2.293598
Epoch 121, loss: 2.297038
Epoch 122, loss: 2.293733
Epoch 123, loss: 2.290701
Epoch 124, loss: 2.288936
Epoch 125, loss: 2.293118
Epoch 126, loss: 2.291141
Epoch 127, loss: 2.292247
Epoch 128, loss: 2.287575
Epoch 129, loss: 2.293688
Epoch 130, loss: 2.297562
Epoch 131, loss: 2.291889
Epoch 132, loss: 2

...

Epoch 27, loss: 2.302779
Epoch 28, loss: 2.302404
Epoch 29, loss: 2.302156
Epoch 30, loss: 2.301442
Epoch 31, loss: 2.303045
Epoch 32, loss: 2.302002
Epoch 33, loss: 2.301812
Epoch 34, loss: 2.301717
Epoch 35, loss: 2.302551
Epoch 36, loss: 2.302585
Epoch 37, loss: 2.302269
Epoch 38, loss: 2.301771
Epoch 39, loss: 2.300735
Epoch 40, loss: 2.302346
Epoch 41, loss: 2.302607
Epoch 42, loss: 2.300788
Epoch 43, loss: 2.302564
Epoch 44, loss: 2.301735
Epoch 45, loss: 2.301070
Epoch 46, loss: 2.302337
Epoch 47, loss: 2.302205
Epoch 48, loss: 2.301497
Epoch 49, loss: 2.302118
Epoch 50, loss: 2.302173
Epoch 51, loss: 2.301563
Epoch 52, loss: 2.302336
Epoch 53, loss: 2.301888
Epoch 54, loss: 2.301976
Epoch 55, loss: 2.302062
Epoch 56, loss: 2.301182
Epoch 57, loss: 2.301431
Epoch 58, loss: 2.301831
Epoch 59, loss: 2.302202
Epoch 60, loss: 2.301916
Epoch 61, loss: 2.302436
Epoch 62, loss: 2.301269
Epoch 63, loss: 2.301454
Epoch 64, loss: 2.300942
Epoch 65, loss: 2.302068
Epoch 66, loss: 2.302143


Epoch 150, loss: 2.300558
Epoch 151, loss: 2.300871
Epoch 152, loss: 2.300975
Epoch 153, loss: 2.301611
Epoch 154, loss: 2.299855
Epoch 155, loss: 2.299448
Epoch 156, loss: 2.301371
Epoch 157, loss: 2.300366
Epoch 158, loss: 2.301536
Epoch 159, loss: 2.300818
Epoch 160, loss: 2.300231
Epoch 161, loss: 2.302270
Epoch 162, loss: 2.300248
Epoch 163, loss: 2.299954
Epoch 164, loss: 2.300716
Epoch 165, loss: 2.300712
Epoch 166, loss: 2.300459
Epoch 167, loss: 2.300851
Epoch 168, loss: 2.300836
Epoch 169, loss: 2.300611
Epoch 170, loss: 2.300576
Epoch 171, loss: 2.300908
Epoch 172, loss: 2.302335
Epoch 173, loss: 2.299160
Epoch 174, loss: 2.301224
Epoch 175, loss: 2.301542
Epoch 176, loss: 2.301276
Epoch 177, loss: 2.301731
Epoch 178, loss: 2.301150
Epoch 179, loss: 2.300406
Epoch 180, loss: 2.300314
Epoch 181, loss: 2.302174
Epoch 182, loss: 2.301823
Epoch 183, loss: 2.300530
Epoch 184, loss: 2.301962
Epoch 185, loss: 2.301197
Epoch 186, loss: 2.301306
Epoch 187, loss: 2.301613
Epoch 188, l

Epoch 73, loss: 2.303903
Epoch 74, loss: 2.303128
Epoch 75, loss: 2.302589
Epoch 76, loss: 2.302900
Epoch 77, loss: 2.301849
Epoch 78, loss: 2.302601
Epoch 79, loss: 2.302033
Epoch 80, loss: 2.303169
Epoch 81, loss: 2.302563
Epoch 82, loss: 2.302650
Epoch 83, loss: 2.301659
Epoch 84, loss: 2.302382
Epoch 85, loss: 2.302002
Epoch 86, loss: 2.302736
Epoch 87, loss: 2.302714
Epoch 88, loss: 2.301942
Epoch 89, loss: 2.302416
Epoch 90, loss: 2.302344
Epoch 91, loss: 2.302892
Epoch 92, loss: 2.302257
Epoch 93, loss: 2.302061
Epoch 94, loss: 2.303412
Epoch 95, loss: 2.302979
Epoch 96, loss: 2.302132
Epoch 97, loss: 2.303076
Epoch 98, loss: 2.302537
Epoch 99, loss: 2.302056
Epoch 100, loss: 2.302297
Epoch 101, loss: 2.301660
Epoch 102, loss: 2.302482
Epoch 103, loss: 2.300631
Epoch 104, loss: 2.302348
Epoch 105, loss: 2.302219
Epoch 106, loss: 2.303275
Epoch 107, loss: 2.301581
Epoch 108, loss: 2.302185
Epoch 109, loss: 2.302047
Epoch 110, loss: 2.303522
Epoch 111, loss: 2.302374
Epoch 112, lo

Epoch 194, loss: 2.302832
Epoch 195, loss: 2.302628
Epoch 196, loss: 2.303801
Epoch 197, loss: 2.302031
Epoch 198, loss: 2.301491
Epoch 199, loss: 2.302215
Epoch 0, loss: 2.302334
Epoch 1, loss: 2.302701
Epoch 2, loss: 2.302262
Epoch 3, loss: 2.302441
Epoch 4, loss: 2.302488
Epoch 5, loss: 2.303029
Epoch 6, loss: 2.302544
Epoch 7, loss: 2.302083
Epoch 8, loss: 2.301492
Epoch 9, loss: 2.302225
Epoch 10, loss: 2.302910
Epoch 11, loss: 2.303062
Epoch 12, loss: 2.302672
Epoch 13, loss: 2.302936
Epoch 14, loss: 2.302887
Epoch 15, loss: 2.302720
Epoch 16, loss: 2.302265
Epoch 17, loss: 2.301662
Epoch 18, loss: 2.302814
Epoch 19, loss: 2.301930
Epoch 20, loss: 2.302264
Epoch 21, loss: 2.302632
Epoch 22, loss: 2.302914
Epoch 23, loss: 2.302486
Epoch 24, loss: 2.302265
Epoch 25, loss: 2.301683
Epoch 26, loss: 2.302836
Epoch 27, loss: 2.302299
Epoch 28, loss: 2.302314
Epoch 29, loss: 2.302761
Epoch 30, loss: 2.302560
Epoch 31, loss: 2.302245
Epoch 32, loss: 2.302866
Epoch 33, loss: 2.302191
Epoc

Epoch 119, loss: 2.303117
Epoch 120, loss: 2.301838
Epoch 121, loss: 2.302623
Epoch 122, loss: 2.302848
Epoch 123, loss: 2.303621
Epoch 124, loss: 2.302916
Epoch 125, loss: 2.302914
Epoch 126, loss: 2.301820
Epoch 127, loss: 2.301877
Epoch 128, loss: 2.301860
Epoch 129, loss: 2.303648
Epoch 130, loss: 2.302374
Epoch 131, loss: 2.302658
Epoch 132, loss: 2.302853
Epoch 133, loss: 2.303160
Epoch 134, loss: 2.302669
Epoch 135, loss: 2.301655
Epoch 136, loss: 2.302714
Epoch 137, loss: 2.303434
Epoch 138, loss: 2.301848
Epoch 139, loss: 2.303289
Epoch 140, loss: 2.302972
Epoch 141, loss: 2.302158
Epoch 142, loss: 2.303160
Epoch 143, loss: 2.303741
Epoch 144, loss: 2.303321
Epoch 145, loss: 2.302649
Epoch 146, loss: 2.302404
Epoch 147, loss: 2.301940
Epoch 148, loss: 2.302996
Epoch 149, loss: 2.302262
Epoch 150, loss: 2.302489
Epoch 151, loss: 2.303087
Epoch 152, loss: 2.303251
Epoch 153, loss: 2.302818
Epoch 154, loss: 2.301481
Epoch 155, loss: 2.302553
Epoch 156, loss: 2.302895
Epoch 157, l

In [14]:
print('parameters: lr = {}, rs = {}'.format(lr, rs))
print('best validation accuracy achieved: %f' % best_val_accuracy)

parameters: lr = 1e-06, rs = 1e-07
best validation accuracy achieved: 0.231000


# Какой же точности мы добились на тестовых данных?

In [15]:
test_pred = best_classifier.predict(test_X)
test_accuracy = multiclass_accuracy(test_pred, test_y)
print('Linear softmax classifier test set accuracy: %f' % (test_accuracy, ))

Linear softmax classifier test set accuracy: 0.194000


**Вывод**: Точность на тестовых данных оказалась ниже, чем на валидационных. Это ожидаемо, поскольку мы обучаемся к валидационной выборке. Чтобы увеличить точность данной модели, можно добавить больше данных. Например, объединить обучающую выборку и валидационную, и обучить с полученными гиперпараметрами.